In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(400_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0329 17:16:04.341056402    1034 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0329 17:16:06.594510324    1034 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
pbar = tqdm()

0it [00:00, ?it/s]

In [3]:
analysis_date = dt.date(2022, 3, 23) ##wednesday, new tables
lbt_itp_id = 170

In [4]:
%%capture
rt_day = rt.OperatorDayAnalysis(lbt_itp_id, analysis_date, pbar)

In [5]:
%%capture
rt_day.set_filter(start_time='15:00', end_time='19:00')
m = rt_day.segment_speed_map(how='low_speeds', size = [1300, 700])

In [20]:
# m

#### siuba query in utils.get_routelines for reference
```
    shapes = (tbl.views.gtfs_schedule_dim_shapes_geo()
          >> filter(_.calitp_extracted_at <= analysis_date, 
                    _.calitp_deleted_at > analysis_date
                   )
          >> filter(_.calitp_itp_id == itp_id)
          >> select(_.calitp_itp_id, _.calitp_url_number, _.calitp_extracted_at,
                    _.calitp_deleted_at, _.shape_id, _.pt_array)
          >> collect()
         )
```

In [7]:
lines = utils.get_routelines(lbt_itp_id, analysis_date).dropna()

/opt/conda/lib/python3.9/site-packages/pybigquery/sqlalchemy_bigquery.py:879: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


In [8]:
lines['num_points'] = lines.geometry.apply(lambda x: len([pt for pt in x.coords]))

In [9]:
lines['has_trip'] = lines.shape_id.isin(rt_day.rt_trips.shape_id)

In [10]:
lines >> filter(_.num_points < 50) >> arrange(_.num_points)

,calitp_itp_id,calitp_url_number,calitp_extracted_at,calitp_deleted_at,shape_id,geometry,num_points,has_trip


In [11]:
a_shape = lines >> filter(_.shape_id == '1720090')

In [12]:
a_shape

,calitp_itp_id,calitp_url_number,calitp_extracted_at,calitp_deleted_at,shape_id,geometry,num_points,has_trip
82,170,0,2022-02-02,2099-01-01,1720090,"LINESTRING (167579.235 -470295.935, 167588.400...",857,True


In [19]:
lines >> count(_.shape_id) >> arrange(_.n)

,shape_id,n
0,10074,1
1,10075,1
2,10078,1
3,1010086,1
4,1010087,1
...,...,...
111,940155,1
112,940156,1
113,940157,1
114,940158,1
